<a href="https://colab.research.google.com/github/nnilayy/Polygon2Mask/blob/main/Polygon2Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip -q /content/balls.zip -d /content/

In [ ]:
import numpy as np
from skimage import draw

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Extracting the shape of the images

In [ ]:
from google.colab.patches import cv2_imshow
import glob
import os
import numpy as np
import cv2

images=sorted(glob.glob("/content/balls/*"))

image_shapes=[]
for image in images:
  image=cv2.imread(image)
  image=np.array(image)
  width,height,_=image.shape
  cv2_imshow(image)
  image_shapes.append((width,height))

names=[]
for image in images:
  basename=os.path.basename(image)
  basename=basename.split('.')[0]
  names.append(basename)

## Extracting the X_coordinates and y_coordinates of all images 

In [ ]:
import json
with open('/content/balls.json', 'r') as f:
  file = json.load(f)
x=[]
y=[]
for i in file:
 x.append(file[str(i)]["regions"]["0"]["shape_attributes"]['all_points_x'])
 y.append(file[str(i)]["regions"]["0"]["shape_attributes"]['all_points_y'])
print(x)
print(y)

## Drawing masks

In [ ]:
def mask(x, y, shape):
    row_fill, col_fill = draw.polygon(x, y, shape)
    mask = np.zeros(shape, dtype=np.bool)
    mask[row_fill,col_fill] = True
    return mask*255

In [ ]:
!mkdir "/content/Bitmaps"

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import os
img=[]
for i, name, shape in zip( range(len(x)), names, image_shapes):
  bw=mask(y[i],x[i],shape)
  img.append(bw)
  cv2.imwrite("/content/Bitmaps/"+str(name)+".jpeg",bw)
  cv2_imshow(bw)

In [ ]:
import matplotlib.pyplot as plt
X=sorted(glob.glob("/content/balls/*"))
y=sorted(glob.glob("/content/Bitmaps/*"))
index=5
image=cv2.imread(X[index])
mask=cv2.imread(y[index])
plt.subplot(1, 2, 1)
plt.imshow(image,cmap='gray')
plt.subplot(1, 2, 2)
plt.imshow(mask,cmap='gray')
plt.show()


## Setting Data Generator For Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
datagen=ImageDataGenerator(
    rotation_range=360,
    # brightness_range=(0,5),
    zoom_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest",
)

## Augmenting Images

In [ ]:
!mkdir "/content/drive/MyDrive/Balls"
!mkdir "/content/drive/MyDrive/Balls/Images"
!mkdir "/content/drive/MyDrive/Balls/Masks"

In [ ]:
images=sorted(glob.glob("/content/balls/*"))
masks=sorted(glob.glob("/content/Bitmaps/*"))
seed=42

increment=0
for image,mask in zip(images,masks):
  img=load_img(image)
  x=img_to_array(img)
  x=x.reshape((1,)+x.shape) 
  i=1
  for batch in datagen.flow(x,batch_size=1,save_to_dir="/content/drive/MyDrive/Balls/Images/",save_prefix=str(increment),save_format="jpg",seed=seed):
    i+=1
    if i>20:
      break
  
  msk=load_img(mask)
  y=img_to_array(msk)
  y=y.reshape((1,)+y.shape)
  i=1
  for batch in datagen.flow(y,batch_size=1,save_to_dir="/content/drive/MyDrive/Balls/Masks/",save_prefix=str(increment),save_format="jpg",seed=seed):
    i+=1
    if i>20:
      break

  increment=increment+1